In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import statistics as st
from collections import Counter
import os
import os.path
import scipy
import csv
from scipy import signal
import pandas as pd

In [47]:
# Create time points
Tmin, Tmax, dt = 0, 1000, 0.025  # Step size
T = np.arange(Tmin, Tmax + dt, dt)

# Declare the current to be used
I_ext = 150

## Nine parameters for IM model
# Axo_Axonic
k, a, b, d, C, vr, vt, c, vpeak = 3.961462878, 0.004638608, 8.683644937, 15, 165, -57.09978287, \
                                  -51.71875628, -73.96850421, 27.79863559

preNeuronType = "Axo-Axonic"

## Make a state vector that has a (v, u) pair for each timestep
s = np.zeros((len(T), 2))

# Create a vector to store spike times
spike_times = np.array([])
isi_mode_list = np.array([])


## Initial values
s[0, 0] = vr
s[0, 1] = 0


# Note that s1[0] is v, s1[1] is u. This is Izhikevich equation in vector form
def s_dt(s1, I):
  v_dt = (k*(s1[0] - vr)*(s1[0] - vt) - s1[1] + I)*(1/C)
  u_dt = a*(b*(s1[0] - vr) - s1[1])
  return np.array([v_dt, u_dt])


## SIMULATE
for t in range(len(T)-1):
  # Calculate the four constants of Runge-Kutta method
  k_1 = s_dt(s[t], I_ext)
  k_2 = s_dt(s[t] + 0.5*dt*k_1, I_ext)
  k_3 = s_dt(s[t] + 0.5*dt*k_2, I_ext)
  k_4 = s_dt(s[t] + dt*k_3, I_ext)

  s[t+1] = s[t] + (1.0/6)*dt*(k_1 + 2*k_2 + 2*k_3 + k_4)

  # Reset the neuron if it has spiked
  if s[t+1, 0] >= vpeak:
    s[t, 0]   = vpeak # add Dirac pulse for visualisation
    s[t+1, 0] = c   # Reset to resting potential
    s[t+1, 1] += d  # Update recovery variable
    spike_times = np.append(spike_times, math.ceil(t*dt))

v = s[:, 0]
u = s[:, 1]


## Nine parameters for IM model
# Pyramidal
postNeuronType = "Pyramidal"
a, b, c, d, k, vr, vt, C, vpeak = 0.00838350334098279, -42.5524776883928, -38.8680990294091, 588.0, \
                                  0.792338703789581, -63.2044008171655, -33.6041733124267, 366.0, 35.8614648558726

# Synaptic Parameters and equations
synaptic_event_times = list(spike_times)

# TPM parameters
g0, tau_d, tau_f, tau_r, Utilization, e_rev = 3.644261648, 10.71107251, 17.20004939, 435.8103009, 0.259914361, -70

tm_model = 'Carlsim'#'Keivan'

if tm_model == 'Keivan':
    # in CARLsim there is no change to the g param from the value it is set as in connect()
    # Note: NS is unclear why this calculation exists and is related to eq. 13 in (Moradi, 2022) supp mat.
    g0 /= Utilization

def synaptic_event(delta_t_, g, g0, tau_d_, tau_r_, tau_f_, utilization, x0, y0, u0, e_syn):
    if tm_model != 'Carlsim':
        # TM Model that depends on tau_d
        tau1r = tau_d_ / ((tau_d_ - tau_r_) if tau_d_ != tau_r_ else 1e-13)
        y_ = y0 * math.exp(-delta_t_ / tau_d_)
        x_ = 1 + (x0 - 1 + tau1r * y0) * math.exp(-delta_t_ / tau_r_) - tau1r * y_
        u_ = u0 * math.exp(-delta_t_ / tau_f_)
        u0 = u_ + utilization * (1 - u_)
        y0 = y_ + u0 * x_
        x0 = x_ - u0 * x_
        g  = g0 * y0
        #print("%f  = %f * %f; u0:%f x0:%f x_:%f" % (g, g0, y0, u0, x0, x_))
    else:
        # Carlsim's TM Model
        A  = 1 / Utilization # this seems to serve the same function as g0 /= Utilization would
        u_ = u0 + ((-u0/tau_f_) + utilization * (1 - u0))
        x_ = x0 + ((1 - x0)/tau_r_ - u_ * x0)
        #print("%f + %f * (%f * %f * %f)" % (g, g0, A, u_, x_))        
        g  = g + g0 * (A * u_ * x0)
        u0 = u_
        x0 = x_
        
    return g, x0, y0, u0


def synaptic_current(I, g, delta_t_, tau_d_, e_syn_, tm_model):
    if tm_model == 'Keivan':
        #g = g * math.exp(-delta_t_ / tau_d_) * e_syn_
        I = g * math.exp(-delta_t_ / tau_d_) * e_syn_
    if tm_model == 'Carlsim':
        I = g * e_syn_
        #print("%f * (1 - (1 / %f))" % (g,tau_d))
    return I

def synaptic_decay(g, tau_d_, tm_model):
    g = g * (1 - (1 / tau_d_))
    return g

def stp_variable_update(u, x, tau_f, tau_r, tm_model):
    tau_f_inv = 1/tau_f
    tau_r_inv = 1/tau_r
    u = u * (1 - tau_f_inv)
    x = x + (1 - x) * tau_r_inv
    #u = u + -u/tau_f
    #x = x + (1 - x)/tau_r
    return u, x

# Initialize synaptic state variables
g, x0, y0 = 0.0, 1.0, 0.0
I = 0.0
if tm_model == 'Carlsim':
    u0 = 0 #Utilization # NS does not see evidence in CARLsim that this is initialized as other than 0
else:
    u0 = 0

# Make a state vector that has a (v, u) pair for each timestep
s = np.zeros((len(T), 2))

# Initial Izhikevich state variables
s[0, 0] = vr
s[0, 1] = 0


# Note that s1[0] is v, s1[1] is u. This is Izhikevich equation in vector form
def s_dt(s1, I):
    v_dt = (k * (s1[0] - vr) * (s1[0] - vt) - s1[1] + I) * (1 / C)
    u_dt = a * (b * (s1[0] - vr) - s1[1])
    return np.array([v_dt, u_dt])

# SIMULATE
next_synaptic_event_time, delta_t, I_syn = synaptic_event_times[0], 0.0, [0]
synaptic_event_time = next_synaptic_event_time
spike_times = np.array([])
for t in range(len(T) - 1):
    v = s[t, 0]
    e_syn = v - e_rev
    time = T[t]

    # in CARLsim this occurs before the synaptic spike current update. doSTPUpdateAndDecayCond() is before globalStateUpdate().
    if (tm_model == 'Carlsim' and t % (1/dt) == 0): # run every whole number millisecond
        g = synaptic_decay(g, tau_d, tm_model)
        u0, x0 = stp_variable_update(u0, x0, tau_f, tau_r, tm_model)

    if next_synaptic_event_time <= time:
        inter_event_time = next_synaptic_event_time - synaptic_event_time
        g, x0, y0, u0 = synaptic_event(inter_event_time, g, g0, tau_d, tau_r, tau_f, Utilization, x0, y0, u0, e_syn)
        # NS addition: shouldn't g be a rolling total rather than equal to g here? Such would accomidate past g from prior spikes?
        # NS updated g for carlsim calcs
        #print(g, time, synaptic_event_time, next_synaptic_event_time)
        print("t:%d I:%.3f\tg:%.3f\tu:%.3f\tx:%.3f\tv:%.3f\tsynaptic spike" % (time,I,g,u0,x0,e_syn))
        synaptic_event_time = time
        if len(synaptic_event_times) > 1:
            del synaptic_event_times[0]
            next_synaptic_event_time = synaptic_event_times[0]
        else:
            next_synaptic_event_time = math.inf

    delta_t = time - synaptic_event_time
    if delta_t >= 0:
        if (t % (1/dt) == 0):
            I = synaptic_current(I, g, delta_t, tau_d, e_syn, tm_model)
            print("t:%d I:%.3f\tg:%.3f\tu:%.3f\tx:%.3f\tv:%.3f\tcurrent decay" % (t/(1/dt),I,g,u0,x0,e_syn))
    I_syn.append(I)

    # Calculate the four constants of Runge-Kutta method
    k_1 = s_dt(s[t], -I)
    k_2 = s_dt(s[t] + 0.5 * dt * k_1, -I)
    k_3 = s_dt(s[t] + 0.5 * dt * k_2, -I)
    k_4 = s_dt(s[t] + dt * k_3, -I)

    s[t + 1] = s[t] + (1.0 / 6) * dt * (k_1 + 2 * k_2 + 2 * k_3 + k_4)

    # Reset the neuron if it has spiked
    if s[t + 1, 0] >= vpeak:
        s[t, 0] = vpeak  # add Dirac pulse for visualisation
        s[t + 1, 0] = c  # Reset to resting potential
        s[t + 1, 1] += d  # Update recovery variable
        spike_times = np.append(spike_times, math.floor(t * dt))

v = s[:, 0]
u = s[:, 1]

# First define function to flip the sign of the current
def sign(lst): 
    return [ -i for i in lst ]

# Compare to CARLsim simulation output
FH = np.loadtxt("HC_IM_05_26_aac_pyr_I_150pA_fast_1_slow_0.txt")
I = FH[1::2]
V = FH[0::2]
I = sign(I)
I = I[0:len(I_syn)-1]
V = V[0:len(I_syn)-1]
ax1 = plt.subplot(211)
ax1.plot(T,np.append(V, [V[-1]]), label = "CARLsim TM") # added the last value of V to ensure the same length as time vector
plt.ylabel('Membrane potential (mV)')
ax2 = plt.subplot(212)
ax2.plot(T,np.append(I, [I[-1]]), label = "CARLsim TM") # added the last value of I to ensure the same length as time vector
plt.ylabel('Synaptic Current (pA)')
plt.xlabel('Time (ms)')
plt.legend(loc = "center right")
plt.tight_layout()

## Plot the membrane potential
ax1 = plt.subplot(211)
ax1.plot(T, v, color = "orange", linestyle='dotted', label = "Keivan TM", alpha=0.85)
plt.ylabel('Membrane potential (mV)')
plt.title(f"{postNeuronType}")
ax2 = plt.subplot(212)
ax2.plot(T, I_syn, color = "orange", linestyle='dotted', label = "Keivan TM", alpha=0.85)
plt.ylabel('Synaptic Current (pA)')
plt.xlabel('Time (ms)')
plt.legend(loc = "center right")
plt.tight_layout()
fileOutputName = preNeuronType + '_' + postNeuronType + '_' + str(I_ext) + 'pA' + \
      '_CARLsim_vs_Keivan_superimposed.png'
plt.savefig(fileOutputName, dpi=800)
plt.clf()


# Look at the error between CARLsim and python computed synaptic signal
I = np.append(I, [I[-1]])
V = np.append(V, [V[-1]])

af = scipy.fft.fft(I_syn)
bf = scipy.fft.fft(I)
c = scipy.ifft(af * scipy.conj(bf))
time_shift = np.argmax(abs(c))
#         print(time_shift)

I_2 = I[:-1]
I_2 = I_2[::int(1/dt)]
I_syn_2 = I_syn[0+time_shift:]
I_syn_2 = I_syn_2[::int(1/dt)]
I_syn_2 = np.array(np.array(I_syn_2,dtype=np.float32))
V_2 = V[:-1]
V_2 = V_2[::int(1/dt)]
v_2 = v[0+time_shift:]
v_2 = v_2[::int(1/dt)]

if len(V_2) == len(v_2):
    ax1 = plt.subplot(211)
    ax1.plot(abs(V_2 - v_2))
    plt.ylabel('Membrane potential (mV)')
    plt.title(f"{postNeuronType}")
    ax2 = plt.subplot(212)
    ax2.plot(abs(I_2 - I_syn_2))
    plt.ylabel('I_syn diff (HCO - Carlsim) (pA)')
    plt.xlabel('Time (ms)')
    plt.tight_layout()
    fileOutputName = preNeuronType + '_' + postNeuronType + '_' + str(I_ext) + 'pA' + \
          '_CARLsim_vs_Keivan_error.png'
    plt.savefig(fileOutputName, dpi=800)
    plt.clf()

    # Append min, max, mean, and median of errors between V and I
    pctErrorV = abs((V_2 - v_2)/v_2)
    pctErrorI = abs((I_2 - I_syn_2)/I_syn_2)
    maxErrorV = max(pctErrorV[~np.isnan(pctErrorV)])
    maxErrorI = max(pctErrorI[~np.isnan(pctErrorI)])
    minErrorV = min(pctErrorV[~np.isnan(pctErrorV)])
    minErrorI = min(pctErrorI[~np.isnan(pctErrorI)])
    meanErrorV = np.mean(pctErrorV[~np.isnan(pctErrorV)])
    meanErrorI = np.mean(pctErrorI[~np.isnan(pctErrorI)])
    medianErrorV = np.median(pctErrorV[~np.isnan(pctErrorV)])
    medianErrorI = np.median(pctErrorI[~np.isnan(pctErrorI)])
    mismatchV = sum(abs(V_2-v_2)*dt)/sum(abs((V_2 + v_2)/2))
    mismatchI = sum(abs(I_2-I_syn_2)*dt)/sum(abs((I_2 + I_syn_2)/2))


t:15 I:0.000	g:3.644	u:0.260	x:0.740	v:6.796	synaptic spike
t:15 I:24.765	g:3.644	u:0.260	x:0.740	v:6.796	current decay
t:16 I:22.236	g:3.304	u:0.245	x:0.741	v:6.730	current decay
t:17 I:19.996	g:2.996	u:0.231	x:0.741	v:6.675	current decay
t:18 I:18.005	g:2.716	u:0.217	x:0.742	v:6.630	current decay
t:19 I:16.232	g:2.462	u:0.205	x:0.742	v:6.592	current decay
t:20 I:14.648	g:2.232	u:0.193	x:0.743	v:6.561	current decay
t:21 I:13.230	g:2.024	u:0.181	x:0.744	v:6.536	current decay
t:22 I:11.958	g:1.835	u:0.171	x:0.744	v:6.517	current decay
t:23 I:10.816	g:1.664	u:0.161	x:0.745	v:6.501	current decay
t:24 I:9.789	g:1.508	u:0.152	x:0.745	v:6.490	current decay
t:25 I:8.863	g:1.368	u:0.143	x:0.746	v:6.481	current decay
t:26 I:8.029	g:1.240	u:0.134	x:0.747	v:6.475	current decay
t:27 I:7.275	g:1.124	u:0.127	x:0.747	v:6.472	current decay
t:28 I:6.594	g:1.019	u:0.119	x:0.748	v:6.470	current decay
t:29 I:5.979	g:0.924	u:0.112	x:0.748	v:6.470	current decay
t:30 I:5.422	g:0.838	u:0.106	x:0.749	v:6.472	c

t:506 I:0.174	g:0.061	u:0.045	x:0.275	v:2.861	current decay
t:507 I:0.157	g:0.055	u:0.042	x:0.277	v:2.847	current decay
t:508 I:0.142	g:0.050	u:0.040	x:0.279	v:2.832	current decay
t:509 I:0.128	g:0.045	u:0.037	x:0.280	v:2.817	current decay
t:510 I:0.115	g:0.041	u:0.035	x:0.282	v:2.802	current decay
t:511 I:0.104	g:0.037	u:0.033	x:0.283	v:2.787	current decay
t:512 I:0.094	g:0.034	u:0.031	x:0.285	v:2.772	current decay
t:513 I:0.085	g:0.031	u:0.029	x:0.287	v:2.757	current decay
t:514 I:0.076	g:0.028	u:0.028	x:0.288	v:2.741	current decay
t:515 I:0.069	g:0.025	u:0.026	x:0.290	v:2.726	current decay
t:516 I:0.062	g:0.023	u:0.025	x:0.292	v:2.711	current decay
t:517 I:0.056	g:0.021	u:0.023	x:0.293	v:2.695	current decay
t:518 I:0.050	g:0.019	u:0.022	x:0.295	v:2.680	current decay
t:519 I:0.045	g:0.017	u:0.020	x:0.297	v:2.664	current decay
t:520 I:0.041	g:0.015	u:0.019	x:0.298	v:2.648	current decay
t:521 I:0.037	g:0.014	u:0.018	x:0.300	v:2.633	current decay
t:522 I:0.033	g:0.013	u:0.017	x:0.301	v:

/tmp/ipykernel_158441/55680725.py:241: DeprecationWarning: scipy.conj is deprecated and will be removed in SciPy 2.0.0, use numpy.conj instead
  c = scipy.ifft(af * scipy.conj(bf))
/tmp/ipykernel_158441/55680725.py:241: DeprecationWarning: scipy.ifft is deprecated and will be removed in SciPy 2.0.0, use scipy.fft.ifft instead
  c = scipy.ifft(af * scipy.conj(bf))
/tmp/ipykernel_158441/55680725.py:272: RuntimeWarning: invalid value encountered in true_divide
  pctErrorI = abs((I_2 - I_syn_2)/I_syn_2)


<Figure size 432x288 with 0 Axes>

In [47]:
# Print stats related to the differences between the voltage computed via Keivan's model and CARLsim
print(maxErrorV)
print(minErrorV)
print(meanErrorV)
print(medianErrorV)
print(mismatchV)

0.0005537050967232892
1.4406427631404644e-06
0.0002935743608628468
0.0003518081076378919
7.5325434774792025e-06


In [46]:
# Print stats related to the differences between the current computed via Keivan's model and CARLsim
print(maxErrorI)
print(minErrorI)
print(meanErrorI)
print(medianErrorI)
print(mismatchI)

1.0
2.3351885965391782e-06
0.022663087706716347
0.01522615489638712
0.00032158528745553755
